In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
test_data=pd.read_csv(r"C:\Users\varun\Downloads\6644a1e287df6_test_data_with_samplefeatures.csv")
batsman_data=pd.read_csv(r"C:\Users\varun\Downloads\663e2b548c98c_batsman_level_scorecard.csv")
bowler_data=pd.read_csv(r"C:\Users\varun\Downloads\663e2b2c60743_bowler_level_scorecard.csv")
match_data=pd.read_csv(r"C:\Users\varun\Downloads\664389efa0868_match_level_scorecard.csv")

In [3]:
test_data['match_dt'] = pd.to_datetime(test_data['match_dt'])
test_data=test_data.sort_values(by='match_dt')
test_data=test_data.reset_index(drop=True)

batsman_data['match_dt'] = pd.to_datetime(batsman_data['match_dt'])
batsman_data=batsman_data.sort_values(by='match_dt')
test_data=test_data.reset_index(drop=True)

bowler_data['match_dt'] = pd.to_datetime(bowler_data['match_dt'])
bowler_data=bowler_data.sort_values(by='match_dt')
bowler_data=bowler_data.reset_index(drop=True)

match_data['match_dt'] = pd.to_datetime(match_data['match_dt'])
match_data=match_data.sort_values(by='match_dt')
match_data=match_data.reset_index(drop=True)

# Batsman

In [4]:
batsman_data.columns

Index(['match id', 'batsman', 'batsman_id', 'batsman_details',
       'is_batsman_captain', 'is_batsman_keeper', 'inning', 'runs',
       'balls_faced', 'over_faced_first', 'wicket kind', 'out_by_bowler',
       'out_by_fielder', 'bowler_id', 'bowler_details', 'is_bowler_keeper',
       'is_bowler_captain', 'strike_rate', 'Fours', 'Sixes', 'match_dt'],
      dtype='object')

In [5]:
batsman_data.describe()

,match id,batsman_id,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,bowler_id,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes
count,2.448300e+04,2.448300e+04,24483.000000,24483.000000,24483.000000,24483.000000,24483.000000,24483.000000,1.812800e+04,18698.0,18128.000000,24474.000000,14200.000000,8994.000000
mean,9.105542e+06,4.129075e+06,0.113957,0.113589,1.490136,18.296491,14.140383,9.192093,4.368210e+06,0.0,0.056873,110.319366,2.731056,1.951523
std,3.187236e+05,2.435776e+06,0.317765,0.317318,0.499913,20.251578,12.774643,6.785681,2.379752e+06,0.0,0.231607,68.869285,2.142437,1.410311
min,8.587795e+06,3.146400e+04,0.000000,0.000000,1.000000,0.000000,0.000000,1.100000,3.406100e+04,0.0,0.000000,0.000000,1.000000,1.000000
25%,8.778825e+06,2.231928e+06,0.000000,0.000000,1.000000,3.000000,4.000000,2.100000,2.538983e+06,0.0,0.000000,64.290000,1.000000,1.000000
50%,9.085075e+06,3.729970e+06,0.000000,0.000000,1.000000,11.000000,10.000000,8.500000,3.913475e+06,0.0,0.000000,108.330000,2.000000,1.000000
75%,9.407600e+06,6.095508e+06,0.000000,0.000000,2.000000,27.000000,20.000000,15.500000,6.399210e+06,0.0,0.000000,150.000000,4.000000,2.000000
max,9.887863e+06,9.883250e+06,1.000000,1.000000,2.000000,162.000000,75.000000,20.900000,9.888584e+06,0.0,1.000000,600.000000,22.000000,13.000000


In [6]:
date=test_data['match_dt'][0]

In [7]:
filtered_data = batsman_data[batsman_data['match_dt'] < date]
all_batsman_ids = batsman_data['batsman_id'].unique()

# Group by batsman and aggregate statistics
aggregated_data = filtered_data.groupby('batsman_id').agg({
    'runs': 'sum',
    'balls_faced': 'sum',
    'Fours': 'sum',
    'Sixes': 'sum'
}).reset_index()

batsman_details = pd.DataFrame({'batsman_id': all_batsman_ids})
batsman_details = batsman_details.merge(aggregated_data, on='batsman_id', how='left').fillna(0)

# Calculate the strike rate
batsman_details['strike_rate'] = (batsman_details['runs'] / batsman_details['balls_faced']) * 100
batsman_details['strike_rate'] = batsman_details['strike_rate'].replace([float('inf'), -float('inf')], 0)  # Handling division by zero


In [8]:
batsman_details

,batsman_id,runs,balls_faced,Fours,Sixes,strike_rate
0,7907451.0,222.0,156.0,25.0,9.0,142.307692
1,2083409.0,793.0,583.0,69.0,28.0,136.020583
2,36665.0,208.0,171.0,19.0,4.0,121.637427
3,1749075.0,1233.0,774.0,140.0,57.0,159.302326
4,1506098.0,468.0,336.0,61.0,7.0,139.285714
...,...,...,...,...,...,...
2113,9403421.0,0.0,0.0,0.0,0.0,NaN
2114,1636004.0,0.0,0.0,0.0,0.0,NaN
2115,7907458.0,0.0,0.0,0.0,0.0,NaN
2116,9883250.0,0.0,0.0,0.0,0.0,NaN


In [9]:
import pandas as pd

def strike_rate(roster_ids, batsman_details, batsman_data,match_id):
    ID = [int(id.split('.')[0]) for id in roster_ids.split(':')]
    count_sr = 0
    
    for i in ID:
        index1 = batsman_details.index[batsman_details['batsman_id'] == i]
        if not index1.empty:
            index_1 = index1[0]
            if batsman_details.at[index_1, 'strike_rate'] >= 120:
                count_sr += 1
                
            index2=batsman_data[(batsman_data['batsman_id']==i) & (batsman_data['match id']==match_id)].index
            if not index2.empty:
                index_2 = index2[0]
                batsman_details['runs'][index_1]+=batsman_data['runs'][index_2]
                batsman_details['balls_faced'][index_1]+=batsman_data['balls_faced'][index_2]
                batsman_details['Fours'][index_1]+=batsman_data['Fours'][index_2]
                batsman_details['Sixes'][index_1]+=batsman_data['Sixes'][index_2]
                batsman_details['strike_rate'][index_1] = (batsman_details['runs'][index_1] / batsman_details['balls_faced'][index_1]) * 100
    
    return count_sr

In [10]:
#For test_data
for k in range(test_data.shape[0]):
    team1_count_sr = strike_rate(test_data['team1_roster_ids'][k], batsman_details, batsman_data,test_data['match id'][k])
    team2_count_sr = strike_rate(test_data['team2_roster_ids'][k], batsman_details, batsman_data,test_data['match id'][k])

    test_data.at[k, 'PlayersSRgreaterthan120_team1'] = team1_count_sr
    test_data.at[k, 'PlayersSRgreaterthan120_team2'] = team2_count_sr

In [11]:
test_data.describe()

,match id,team1_id,team2_id,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,PlayersSRgreaterthan120_team1,PlayersSRgreaterthan120_team2
count,2.710000e+02,271.000000,271.000000,271.000000,271.000000,271.000000,264.000000,271.000000,261.000000,271.000000,271.000000
mean,9.301500e+06,23274.180812,23105.225092,7369.896679,1.098773,4.379993,160.460540,42.220295,154.293439,4.985240,5.202952
std,2.167870e+05,17577.908875,17773.159020,6620.422797,0.832996,13.643121,14.929308,38.915871,14.253331,2.641507,2.698311
min,8.844338e+06,20.000000,20.000000,83.000000,0.000000,0.009901,81.000000,0.000000,118.500000,0.000000,0.000000
25%,9.094318e+06,7608.000000,6838.000000,1476.000000,0.633971,0.672131,153.000000,0.000000,145.458333,3.000000,3.000000
50%,9.272549e+06,28594.000000,23869.000000,5004.000000,0.888889,1.000000,161.600000,50.000000,154.714286,5.000000,6.000000
75%,9.457496e+06,40361.000000,40438.000000,13530.000000,1.355042,1.952381,170.033333,66.670000,163.300000,7.000000,7.000000
max,9.849650e+06,49657.000000,49657.000000,23841.000000,6.000000,101.000000,195.866667,100.000000,200.285714,11.000000,11.000000


# Bowler

In [12]:
bowler_data.describe()

,match id,bowler_id,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs
count,1.853900e+04,1.853900e+04,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000
mean,9.102037e+06,4.417985e+06,0.063811,0.000054,1.494201,25.084147,0.997896,18.747505,8.367395,0.033605,7.043314,2.100005,0.943632,0.629700,0.081018
std,3.201571e+05,2.444409e+06,0.244423,0.007344,0.499980,10.835197,1.050482,6.221394,3.260218,0.186972,3.840311,1.605844,1.073823,0.937492,0.314390
min,8.587795e+06,3.406100e+04,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.778755e+06,2.533666e+06,0.000000,0.000000,1.000000,17.000000,0.000000,12.000000,6.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000
50%,9.085012e+06,4.001934e+06,0.000000,0.000000,1.000000,25.000000,1.000000,20.000000,8.000000,0.000000,7.000000,2.000000,1.000000,0.000000,0.000000
75%,9.388539e+06,6.487970e+06,0.000000,0.000000,2.000000,32.000000,2.000000,24.000000,10.000000,0.000000,10.000000,3.000000,1.000000,1.000000,0.000000
max,9.887863e+06,9.888584e+06,1.000000,1.000000,2.000000,77.000000,6.000000,25.000000,36.000000,3.000000,24.000000,11.000000,9.000000,8.000000,5.000000


In [13]:
bowler_data.shape[0]

18539

In [14]:
bowler_data.columns

Index(['match id', 'bowler', 'bowler_id', 'bowler_details',
       'is_bowler_captain', 'is_bowler_keeper', 'inning', 'runs',
       'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
       'Sixes', 'wides', 'noballs', 'match_dt'],
      dtype='object')

In [15]:
filtered_data = bowler_data[bowler_data['match_dt'] < date]
all_bowler_ids = bowler_data['bowler_id'].unique()

# Group by batsman and aggregate statistics
aggregated_data = filtered_data.groupby('bowler_id').agg({
    'runs': 'sum',
    'wicket_count': 'sum',
    'balls_bowled': 'sum',
    'Fours': 'sum',
    'Sixes': 'sum',
    'maiden': 'sum',
    'dots': 'sum',
    'wides': 'sum',
    'noballs': 'sum'
}).reset_index()

bowler_details = pd.DataFrame({'bowler_id': all_bowler_ids})
bowler_details = bowler_details.merge(aggregated_data, on='bowler_id', how='left').fillna(0)

# Calculate the strike rate
bowler_details['economy'] = (bowler_details['runs'] / bowler_details['balls_bowled']) * 6
bowler_details['economy'] = bowler_details['economy'].replace([float('inf'), -float('inf')], 0)  # Handling division by zero


In [16]:
bowler_details.describe()

,bowler_id,runs,wicket_count,balls_bowled,Fours,Sixes,maiden,dots,wides,noballs,economy
count,1.541000e+03,1541.000000,1541.000000,1541.000000,1541.000000,1541.000000,1541.000000,1541.000000,1541.000000,1541.000000,947.000000
mean,5.409926e+06,125.038936,5.016872,95.252433,10.427644,4.541207,0.168722,35.851395,3.103829,0.394549,8.462570
std,2.695434e+06,194.714839,8.669980,150.929834,16.885619,7.375928,0.524796,59.294297,6.038910,1.152669,2.607762
min,3.406100e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.102854e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.122359
50%,5.425146e+06,33.000000,1.000000,24.000000,3.000000,1.000000,0.000000,7.000000,0.000000,0.000000,8.000000
75%,8.058840e+06,171.000000,7.000000,130.000000,14.000000,6.000000,0.000000,48.000000,4.000000,0.000000,9.159091
max,9.888584e+06,1342.000000,76.000000,1204.000000,109.000000,48.000000,4.000000,481.000000,64.000000,11.000000,27.000000


In [17]:
import pandas as pd

def economy_rate(roster_ids, bowler_details, bowler_data,match_id):
    ID = [int(id.split('.')[0]) for id in roster_ids.split(':')]
    count_economy = 0
    
    for i in ID:
        index1 = bowler_details.index[bowler_details['bowler_id'] == i]
        if not index1.empty:
            index_1 = index1[0]
            if bowler_details.at[index_1, 'economy'] <=10:
                count_economy += 1
                
            index2=bowler_data[(bowler_data['bowler_id']==i) & (bowler_data['match id']==match_id)].index
            if not index2.empty:
                index_2 = index2[0]
                bowler_details['runs'][index_1]+=bowler_data['runs'][index_2]
                bowler_details['wicket_count'][index_1]+=bowler_data['wicket_count'][index_2]
                bowler_details['balls_bowled'][index_1]+=bowler_data['balls_bowled'][index_2]
                bowler_details['Fours'][index_1]+=bowler_data['Fours'][index_2]
                bowler_details['Sixes'][index_1]+=bowler_data['Sixes'][index_2]
                bowler_details['maiden'][index_1]+=bowler_data['maiden'][index_2]
                bowler_details['dots'][index_1]+=bowler_data['dots'][index_2]
                bowler_details['wides'][index_1]+=bowler_data['wides'][index_2]
                bowler_details['noballs'][index_1]+=bowler_data['noballs'][index_2]
                bowler_details['economy'][index_1] = (bowler_details['runs'][index_1] / bowler_details['balls_bowled'][index_1]) * 6
    
    return count_economy

In [18]:
for k in range(test_data.shape[0]):
    team1_count_economy = economy_rate(test_data['team1_roster_ids'][k], bowler_details, bowler_data,test_data['match id'][k])
    team2_count_economy= economy_rate(test_data['team2_roster_ids'][k], bowler_details, bowler_data,test_data['match id'][k])

    test_data.at[k, 'Playerseconomylessthan10_team1'] = team1_count_economy
    test_data.at[k, 'Playerseconomylessthan10_team2'] = team2_count_economy

In [19]:
test_data.describe()

,match id,team1_id,team2_id,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,PlayersSRgreaterthan120_team1,PlayersSRgreaterthan120_team2,Playerseconomylessthan10_team1,Playerseconomylessthan10_team2
count,2.710000e+02,271.000000,271.000000,271.000000,271.000000,271.000000,264.000000,271.000000,261.000000,271.000000,271.000000,271.000000,271.000000
mean,9.301500e+06,23274.180812,23105.225092,7369.896679,1.098773,4.379993,160.460540,42.220295,154.293439,4.985240,5.202952,5.309963,5.206642
std,2.167870e+05,17577.908875,17773.159020,6620.422797,0.832996,13.643121,14.929308,38.915871,14.253331,2.641507,2.698311,2.042115,2.044349
min,8.844338e+06,20.000000,20.000000,83.000000,0.000000,0.009901,81.000000,0.000000,118.500000,0.000000,0.000000,0.000000,0.000000
25%,9.094318e+06,7608.000000,6838.000000,1476.000000,0.633971,0.672131,153.000000,0.000000,145.458333,3.000000,3.000000,4.000000,4.000000
50%,9.272549e+06,28594.000000,23869.000000,5004.000000,0.888889,1.000000,161.600000,50.000000,154.714286,5.000000,6.000000,6.000000,6.000000
75%,9.457496e+06,40361.000000,40438.000000,13530.000000,1.355042,1.952381,170.033333,66.670000,163.300000,7.000000,7.000000,7.000000,7.000000
max,9.849650e+06,49657.000000,49657.000000,23841.000000,6.000000,101.000000,195.866667,100.000000,200.285714,11.000000,11.000000,10.000000,10.000000


# Match 

In [20]:
match_data.columns

Index(['match id', 'team1', 'team2', 'winner', 'by', 'win amount',
       'toss winner', 'toss decision', 'venue', 'city', 'match_dt', 'lighting',
       'series_name', 'season', 'ground_id', 'umpire1', 'umpire2',
       'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs',
       'inning2_wickets', 'inning2_balls', 'team1_id', 'team1_roster_ids',
       'team2_id', 'team2_roster_ids', 'series_type', 'winner_id',
       'player_of_the_match_id'],
      dtype='object')

In [21]:
match_data

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,...,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,...,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0...,18570,2653993.0:6718326.0:6718382.0:2486896.0:228878...,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,...,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1...,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652....,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,...,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:306369...,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803....,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,...,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0...,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498...,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,...,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948...,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127...,other_domestic,33928,3125849.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1684,9717504,Nw Zd,Bh,Bh,wickets,5.0,Bh,field,Mn Pk Nr,Napier,...,137.0,5.0,114.0,48,6718326.0:4381761.0:2672214.0:5764576.0:306995...,188,3758565.0:2104332.0:4403419.0:3056752.0:693023...,international,188,5831622.0
1685,9866373,Oo,Wn,Wn,wickets,9.0,Oo,bat,Mx Pk Aa,Alexandra,...,53.0,1.0,29.0,18360,2319638.0:8444855.0:4172706.0:7907444.0:631714...,18570,9036516.0:8271969.0:3913447.0:9523954.0:835478...,other_domestic,18570,6718410.0
1686,9702783,Ht Hs,Me Ss,Me Ss,wickets,7.0,Me Ss,field,Be Ol Ht,Hobart,...,67.0,3.0,42.0,33928,1611364.0:5843200.0:5716402.0:3890963.0:194364...,33949,5406540.0:2275195.0:2275097.0:3901078.0:796099...,other_domestic,33949,2275195.0
1687,9702790,Ae Ss,Me Rs,Me Rs,wickets,4.0,Ae Ss,bat,Ds Sm Me,Melbourne,...,178.0,5.0,115.0,33914,4239038.0:2161599.0:2286472.0:5469008.0:368195...,33942,4003390.0:2654014.0:8176356.0:46794.0:1635773....,other_domestic,33942,8176356.0


In [22]:
# Parse the match date column to datetime
match_data['match_dt'] = pd.to_datetime(match_data['match_dt'])

# Initialize an empty list to store the results
team_stats_list = []

# Loop over each unique team
for team_id in pd.concat([match_data['team1_id'], match_data['team2_id']]).unique():
    team_matches = match_data[(match_data['team1_id'] == team_id) | (match_data['team2_id'] == team_id)]
    
    team_runs_wickets = []
    
    for _, row in team_matches.iterrows():
        # Determine if the team is team1 or team2
        is_team1 = row['team1_id'] == team_id
        
        # Determine if the team batted in the first or second inning
        if (is_team1 and row['toss winner'] == row['team1'] and row['toss decision'] == 'bat') or \
           (not is_team1 and row['toss winner'] == row['team2'] and row['toss decision'] == 'bat') or \
           (is_team1 and row['toss winner'] == row['team2'] and row['toss decision'] == 'field') or \
           (not is_team1 and row['toss winner'] == row['team1'] and row['toss decision'] == 'field'):
            inning_runs = row['inning1_runs']
            inning_wickets = row['inning1_wickets']
        else:
            inning_runs = row['inning2_runs']
            inning_wickets = row['inning2_wickets']
        
        team_runs_wickets.append((inning_runs, inning_wickets, row['match_dt']))
    
    team_runs_wickets_df = pd.DataFrame(team_runs_wickets, columns=['runs', 'wickets', 'match_dt'])
    team_runs_wickets_df = team_runs_wickets_df.sort_values(by='match_dt')
    team_runs_wickets_df['cum_avg_runs'] = team_runs_wickets_df['runs'].expanding().mean()
    team_runs_wickets_df['cum_avg_wickets'] = team_runs_wickets_df['wickets'].expanding().mean()
    
    team_stats_list.append((team_id, 
                            team_runs_wickets_df['cum_avg_runs'].iloc[-1], 
                            team_runs_wickets_df['cum_avg_wickets'].iloc[-1]))

# Convert the list to a DataFrame
team_stats_df = pd.DataFrame(team_stats_list, columns=['team_id', 'cum_avg_runs', 'cum_avg_wickets'])

In [23]:
team_stats_df

,team_id,cum_avg_runs,cum_avg_wickets
0,17982,144.000000,6.120000
1,33942,146.875000,7.125000
2,33956,157.657895,5.868421
3,33928,158.382353,6.852941
4,17583,147.869565,7.000000
...,...,...,...
169,28594,141.000000,4.000000
170,1210,114.333333,7.666667
171,1196,126.000000,4.000000
172,1322,106.000000,10.000000


In [24]:
import pandas as pd

# Assuming match_data and train_data are already loaded into DataFrames
# match_data = pd.read_csv('path_to_your_match_data.csv')
# train_data = pd.read_csv('path_to_your_train_data.csv')

# Parse the match date column to datetime
match_data['match_dt'] = pd.to_datetime(match_data['match_dt'])
test_data['match_dt'] = pd.to_datetime(test_data['match_dt'])

# Function to calculate cumulative averages
def calculate_cumulative_averages(team_runs_wickets):
    team_runs_wickets = team_runs_wickets.sort_values(by='match_dt')
    team_runs_wickets['cum_avg_runs'] = team_runs_wickets['runs'].expanding().mean()
    team_runs_wickets['cum_avg_wickets'] = team_runs_wickets['wickets'].expanding().mean()
    return team_runs_wickets

# Create a dictionary from team_stats_df for quick lookup and update
team_stats = team_stats_df.set_index('team_id').T.to_dict()

# Initialize columns for the train_data DataFrame
test_data['team1_avg_runs'] = 0
test_data['team1_avg_wickets'] = 0
test_data['team2_avg_runs'] = 0
test_data['team2_avg_wickets'] = 0

# Iterate through each row in
for i, row in test_data.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_id = row['match id']
    
    # Check if team1_id and team2_id exist in team_stats
    if team1_id in team_stats:
        test_data.at[i, 'team1_avg_runs'] = team_stats[team1_id].get('cum_avg_runs', 0)
        test_data.at[i, 'team1_avg_wickets'] = team_stats[team1_id].get('cum_avg_wickets', 0)
    else:
        test_data.at[i, 'team1_avg_runs'] = 0
        test_data.at[i, 'team1_avg_wickets'] = 0

    if team2_id in team_stats:
        test_data.at[i, 'team2_avg_runs'] = team_stats[team2_id].get('cum_avg_runs', 0)
        test_data.at[i, 'team2_avg_wickets'] = team_stats[team2_id].get('cum_avg_wickets', 0)
    else:
        test_data.at[i, 'team2_avg_runs'] = 0
        test_data.at[i, 'team2_avg_wickets'] = 0
    
    # Update the averages in team_stats based on the match results
    match_row = match_data[match_data['match id'] == match_id]
    if not match_row.empty:
        match_row = match_row.iloc[0]
        
        # Team 1 updates
        if (match_row['toss winner'] == match_row['team1'] and match_row['toss decision'] == 'bat') or \
           (match_row['toss winner'] == match_row['team2'] and match_row['toss decision'] == 'field'):
            team1_runs = match_row['inning1_runs']
            team1_wickets = match_row['inning1_wickets']
        else:
            team1_runs = match_row['inning2_runs']
            team1_wickets = match_row['inning2_wickets']
        
        if team1_id in team_stats:
            team_stats[team1_id]['runs'] = team_stats[team1_id].get('runs', []) + [team1_runs]
            team_stats[team1_id]['wickets'] = team_stats[team1_id].get('wickets', []) + [team1_wickets]
        else:
            team_stats[team1_id] = {'runs': [team1_runs], 'wickets': [team1_wickets]}
        
        # Team 2 updates
        if (match_row['toss winner'] == match_row['team2'] and match_row['toss decision'] == 'bat') or \
           (match_row['toss winner'] == match_row['team1'] and match_row['toss decision'] == 'field'):
            team2_runs = match_row['inning1_runs']
            team2_wickets = match_row['inning1_wickets']
        else:
            team2_runs = match_row['inning2_runs']
            team2_wickets = match_row['inning2_wickets']
        
        if team2_id in team_stats:
            team_stats[team2_id]['runs'] = team_stats[team2_id].get('runs', []) + [team2_runs]
            team_stats[team2_id]['wickets'] = team_stats[team2_id].get('wickets', []) + [team2_wickets]
        else:
            team_stats[team2_id] = {'runs': [team2_runs], 'wickets': [team2_wickets]}
        
        # Recalculate the cumulative averages
        team1_df = pd.DataFrame({'runs': team_stats[team1_id]['runs'], 'wickets': team_stats[team1_id]['wickets'], 'match_dt': [row['match_dt']] * len(team_stats[team1_id]['runs'])})
        team1_df = calculate_cumulative_averages(team1_df)
        team_stats[team1_id]['cum_avg_runs'] = team1_df['cum_avg_runs'].iloc[-1]
        team_stats[team1_id]['cum_avg_wickets'] = team1_df['cum_avg_wickets'].iloc[-1]
        
        team2_df = pd.DataFrame({'runs': team_stats[team2_id]['runs'], 'wickets': team_stats[team2_id]['wickets'], 'match_dt': [row['match_dt']] * len(team_stats[team2_id]['runs'])})
        team2_df = calculate_cumulative_averages(team2_df)
        team_stats[team2_id]['cum_avg_runs'] = team2_df['cum_avg_runs'].iloc[-1]
        team_stats[team2_id]['cum_avg_wickets'] = team2_df['cum_avg_wickets'].iloc[-1]


In [25]:
test_data.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,...,team1_winp_team2_last15,ground_avg_runs_last15,PlayersSRgreaterthan120_team1,PlayersSRgreaterthan120_team2,Playerseconomylessthan10_team1,Playerseconomylessthan10_team2,team1_avg_runs,team1_avg_wickets,team2_avg_runs,team2_avg_wickets
0,8887752,Sy Tr,33963,7572123.0:1749075.0:6718802.0:5788320.0:753465...,Me Rs,33942,5406540.0:37351.0:46794.0:2336473.0:7534687.0:...,Me Rs,field,Ds Sm Me,...,100.00,161.750000,7.0,6.0,7.0,6.0,152,6,146,7
1,8887780,Ht Hs,33928,5843200.0:4223883.0:2340372.0:6381962.0:216159...,Sy Tr,33963,7572123.0:1749075.0:6718802.0:5788320.0:753465...,Ht Hs,bat,Me Ct Gd,...,50.00,158.777778,10.0,7.0,6.0,7.0,158,6,152,6
2,8887794,Ae Ss,33914,4239038.0:5492318.0:5469008.0:1635773.0:555425...,Be Ht,33921,7870043.0:2286472.0:2336613.0:3651472.0:578676...,Ae Ss,bat,Be Ct Gd Wa Be,...,33.33,164.000000,6.0,7.0,4.0,4.0,153,6,153,6
3,8887801,Ht Hs,33928,5843200.0:4223883.0:1611364.0:2161599.0:624925...,Sy Tr,33963,7572123.0:1749075.0:6718802.0:5788320.0:753465...,Ht Hs,bat,Ds Sm Me,...,50.00,161.750000,11.0,7.0,6.0,7.0,158,6,152,6
4,8887745,Ae Ss,33914,4239038.0:5056134.0:2082499.0:1635773.0:546900...,Me Ss,33949,4003390.0:2275195.0:2669316.0:2275097.0:390107...,Me Ss,field,Ae Ol,...,33.33,158.250000,5.0,8.0,5.0,8.0,153,6,147,6


In [26]:
test_data.count()

match id                          271
team1                             271
team1_id                          271
team1_roster_ids                  271
team2                             271
team2_id                          271
team2_roster_ids                  271
toss winner                       271
toss decision                     271
venue                             271
city                              271
match_dt                          271
lighting                          271
series_name                       271
season                            271
ground_id                         271
team_count_50runs_last15          271
team_winp_last5                   271
team1only_avg_runs_last15         264
team1_winp_team2_last15           271
ground_avg_runs_last15            261
PlayersSRgreaterthan120_team1     271
PlayersSRgreaterthan120_team2     271
Playerseconomylessthan10_team1    271
Playerseconomylessthan10_team2    271
team1_avg_runs                    271
team1_avg_wi

In [27]:
test_data['toss winner'] = (test_data['toss winner'] == test_data['team2']).astype('int64')

In [28]:
test_data.drop(columns=[ 'team1', 'team2', 'team1_roster_ids','team2_roster_ids'], inplace=True)

In [29]:
label_encoder = LabelEncoder()

categorical_columns = ['toss decision','venue', 'city', 'match_dt', 'lighting', 'series_name', 'season', 'ground_id']
for col in categorical_columns:
    test_data[col] = label_encoder.fit_transform(test_data[col])

In [30]:
test_data=test_data.drop(columns=['venue', 'city', 'match_dt', 'lighting','series_name', 'season','ground_id'])

In [31]:
test_data=test_data.drop(columns=['toss winner', 'toss decision','team1only_avg_runs_last15','ground_avg_runs_last15'])

In [32]:
test_data.columns

Index(['match id', 'team1_id', 'team2_id', 'team_count_50runs_last15',
       'team_winp_last5', 'team1_winp_team2_last15',
       'PlayersSRgreaterthan120_team1', 'PlayersSRgreaterthan120_team2',
       'Playerseconomylessthan10_team1', 'Playerseconomylessthan10_team2',
       'team1_avg_runs', 'team1_avg_wickets', 'team2_avg_runs',
       'team2_avg_wickets'],
      dtype='object')

In [33]:
from IPython.display import FileLink

# Save DataFrame to Excel file
test_data.to_excel('data.xlsx', index=False)
# Create a link to download the dataframe
FileLink('data.xlsx')

C:\Users\varun\data.xlsx